In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

### 절대 모멘텀
1. 주식데이터 로드
2. 년월 파생변수 생성 -> Date 컬럼에서 년-월 추출해 대입
3. 년,월 별 마지막 날의 month_last_dt 데이터 프레임 생성
4. 전월의 종가를 가지는 파생변수 하나 생성
5. 전년의 종가를 가지는 파생변수 하나 생성
6. 전월, 전년도의 종가를 가지고 거래내역 생성
7. 수익률 계산

In [ ]:
df = pd.read_csv("../csv/GDX.csv", index_col="Date")

In [ ]:
# 변경 전 index 타입 찍어보기
print(df.index)
# index 를 시계열 데이터로 변경
df.index = pd.to_datetime(df.index, format="%Y-%m-%d")

In [ ]:
# 새로운 파생변수 년-월 생성
df["STD-YM"] = list(map(lambda x: datetime.strftime(x,"%Y-%m"),df.index))
df.head(10)


# df["Date"]를 인덱스로 바꾸지 않았더라면(컬럼이었으면)
# df["Date"].dt.strftime("%Y-%m")으로 변경해서 새로운 파생변수에 담아주면 됨

In [ ]:
# 월별 마지막 인덱스만 추출해 새로운 데이터프레임에 대입

# case1 - shift() 이용
# 현재의 STD-YM 값과 다음 행의 STD-YM 값이 다른 경우
# -> df["STD-YM"]!=df["STD-YM"].shift(-1)
# 위의 데이터들을 loc[]의 조건으로 넣으면서 해당 행의 STD-YM 값이 True인 행만 추출
monthly_df = df.loc[df["STD-YM"]!=df["STD-YM"].shift(-1)]
monthly_df

In [ ]:
# case2 -> for문 이용
month_list_df = pd.DataFrame()

# for문 돌기 위해 df["STD-YM"] 행 정보의 중복값을 제거하고 원자값으로 구성
_list = df["STD-YM"].unique()

for i in _list:
    # 년-월 별 데이터프레임들로 쪼갠 후 가장 마지막 행을 month_list_df에 추가
    last_df = df.loc[df["STD-YM"] == i].tail(1)
    month_list_df = pd.concat([month_list_df,last_df])

month_list_df

In [ ]:
# 전 월의 종가를 가지는 파생변수(BF_1M)을 생성하고 결측치는 0으로 대체
# 전 년도의 종가를 가지는 파생변수(BF_12M)을 생성하고 결측치는 0으로 대체

month_list_df["BF_1M"] = month_list_df["Adj Close"].shift(1).fillna(0)
month_list_df["BF_12M"] = month_list_df["Adj Close"].shift(12).fillna(0)

# month_list_df 의 0,12번째 행 보기
month_list_df.iloc[[0,12]]

In [ ]:
# 거래내역 추가
# 거래 조건) (전 월 종가 / 전 년도 종가) - 1 값이 0보다 크고 무한대가 아닌 경우

df["trade"] = ""

# month_list_df의 인덱스, 즉 년-월을 기준으로 반복문 실행
for i in month_list_df.index:
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = (month_list_df.loc[i,"BF_1M"] / month_list_df.loc[i,"BF_12M"]) - 1

    # 절대 모멘텀 지표에 따라 True/False로 구분
    flag = True if ((momentum_index > 0) and (momentum_index != -np.inf) and (momentum_index != np.inf)) else False

    if flag:
        signal = "buy"

    print("날짜 : ",i,"모멘텀 인덱스 : ",momentum_index,"flag : ",flag,"signal : ",signal)

    df.loc[i,"trade"] = signal

In [ ]:
df["trade"].value_counts()

### 수익률 계산
1. 파생변수 return 생성 - 1로 초기화
2. rtn=1.0,buy,sell 변수 생성 - 0으로 초기화
3. 반복문 이용해 구매한 시점 뽑기(현재 행의 trade가 buy이고 전 행의 trade가 ""인 경우 구매한 날)
4. 수익률 계산해서 return 파생변수에 대입

### 누적수익률 계산
1. acc_rtn=1.0 변수 생성
2. return의 항목들을 누적곱하여 acc_rtn컬럼에 대입
3. 최종적으로 acc_rtn 출력

In [ ]:
df.loc[df["trade"]=="buy"].index

In [ ]:
df["return"] = 1

rtn=1.0
buy=0
sell=0


for i in df.index:
    if (df.loc[i,"trade"] == "buy" and  df.shift(1).loc[i,"trade"] == ""):
        buy = df.loc[i,"Adj Close"]

        print("진입일 : ",i,"구매가격 : ",buy)
    
    elif (df.loc[i,"trade"] == "" and  df.shift(1).loc[i,"trade"] == "buy"):
        sell = df.loc[i,"Adj Close"]
        rtn = (sell - buy) / buy + 1

        print("판매일 : ",i,"sell : ",sell,"    rtn : ",rtn)

        df.loc[i,"return"] = (sell - buy) / buy + 1

In [ ]:
df

In [ ]:
# # way1)
# # 누적곱 함수
# df["return"].cumprod()


# way2)
# for문 사용해 누적곱
df["acc_rtn"] = 1
acc_rtn = 1

for i in df.index:
    rtn = df.loc[i,"return"]
    acc_rtn *= rtn

    df.loc[i,"acc_rtn"] = acc_rtn

print("누적수익률 : ",acc_rtn)

In [ ]:
# way1)
# 누적곱 함수
df["return"].cumprod()

In [ ]:
df.tail(1)

In [ ]:
df.index.sort_values(ascending=True).value_counts()

### 절대 모멘텀 함수화
1. 첫번째 함수
    - 매개변수 1개(데이터프레임 df1)
    - 데이터프레임 결측치와 이상치 제거
    - 수정종가 제외한 나머지 컬럼 삭제
    - 인덱스 시계열 변경
    - "STD-YM" 파생변수 생성해 인덱스의 년-월 추출해서 대입

2. 두번째 함수
    - 매개변수 1개(데이터프레임)
    - 새로운 데이터프레임(df2) 생성
    - 인자값으로 받아온 데이터프레임에서 년-월별 마지막 데이터 새로운 데이터프레임에 대입
    - df2에 파생변수 2개 생성
        - "BF_1M"(전 월 종가),"BF_12M"(전 년도 종가)
        - 두 파생변수 모두 결측치는 0으로 대체
    - return df2

3. 세번째 함수
    - 매개변수 2개(첫번째 함수 결과, 두번째 함수 결과)
    - df1에 수익률 파생변수 return 생성, 모든 초기값은 1로 설정
    - df2 값들을 이용해 momentum_index를 구하고 df1에 거래내역 삽입
    - df1의 거래내역을 이용해 수익률 return컬럼에 대입
    - return 컬럼의 데이터를 가지고 누적수익률 acc_return 컬럼에 값 채우기
    - 누적수익률을 print 이용해 출력
    - return df1

In [ ]:
def momentum_1(df):
    # 인덱스가 Date가 아닌 경우 -> Date 컬럼을 인덱스로 변경
    if "Date" in df.columns:
        df.set_index("Date", inplace=True)

    df = df.loc[~df.isin((np.nan, np.inf, -np.inf)).any(axis="columns")]
    print(type(df))

    ## 여기서 df1 = df["Adj Close"] 대괄호 하나만 치면
    ## Series 형태로 반환
    ## -> 데이터프레임 사용하려고 할 때 오류남
    df1 = df[["Adj Close"]]
    df1.index = pd.to_datetime(df1.index, format="%Y-%m-%d")

    df1["STD-YM"] = list(map(lambda x: datetime.strftime(x,"%Y-%m"),df1.index))
    print(df1)
    print(type(df1))

    return df1

In [ ]:
def momentum_2(df):
    # # 방법1) -df2> for문 이용
    # df2 = pd.DataFrame()

    # # for문 돌기 위해 df["STD-YM"] 행 정보의 중복값을 제거
    # _list = df["STD-YM"].unique()

    # for i in _list:
    #     # 년-월 별 데이터프레임들로 쪼갠 후 가장 마지막 행을 df2에 추가
    #     last_df = df.loc[df["STD-YM"] == i].tail(1)
    #     df2 = pd.concat([df2,last_df])

    # 방법2)
    df2 = df.loc[df["STD-YM"]!=df["STD-YM"].shift(-1)]

    df2["BF_1M"] = df2["Adj Close"].shift(1).fillna(0)
    df2["BF_12M"] = df2["Adj Close"].shift(12).fillna(0)

    return df2

In [ ]:
def momentum_3(df1,df2):
    # 수익률 계산을 위한 변수
    buy = 0
    sell = 0

    df1["trade"] = ""
    df1["return"] = 1

    for i in df2.index:
        signal = ""
        momentum_index = (df2.loc[i,"BF_1M"] / df2.loc[i,"BF_12M"]) - 1

        flag = True if ((momentum_index>0) and (momentum_index!=np.inf) and (momentum_index!=-np.inf)) else False

        if flag:
            signal = "buy"

        df1.loc[i,"trade"] = signal

    for i in df1.index:
        ## 매수한 경우
        if (df1.loc[i,"trade"] == "buy" and  df1.shift(1).loc[i,"trade"] == ""):
            buy = df1.loc[i,"Adj Close"]
    
        ## 매도한 경우
        elif (df1.loc[i,"trade"] == "" and  df1.shift(1).loc[i,"trade"] == "buy"):
            sell = df1.loc[i,"Adj Close"]
            rtn = (sell - buy) / buy + 1

            df1.loc[i,"return"] = (sell - buy) / buy + 1
    
    df1["acc_rtn"] = df1["return"].cumprod()

    print("누적수익률\n",df1.tail(1)["acc_rtn"])

    return df1


In [ ]:
df = pd.read_csv("../csv/GDX.csv", index_col="Date")
# df = pd.read_csv("../csv/GM.csv")

In [ ]:
_df1 = momentum_1(df)
_df2 = momentum_2(_df1)         
_df3 = momentum_3(_df1,_df2)

#### 날짜형 문자 시계열로 변경하기 변형


In [ ]:
# datetime 모듈 이용해 시계열로 변경하기
from datetime import datetime

# df.index = df.index.apply(lambda x:datetime.strptime(x, "%Y-%m-%d"))
# df.index  ->  data.index(list형태)에는 .apply() 적용 불가
# .apply()는 Series,DataFrame 형태에만 적용 가능
def change_index(x):
    return datetime.strptime(x,"%Y-%m-%d")

df.index = list(map(change_index,df.index))

In [ ]:
test_df = df.copy()
test_df = test_df.reset_index()
test_df["Date"] = test_df["Date"].apply(lambda x:datetime.strptime(x,"%Y-%m-%d"))
test_df.set_index("Date", inplace=True)
test_df.index

In [ ]:
test_df2 = df.copy()

def change_index(x):
    return datetime.strptime(x,"%Y-%m-%d")

test_df2.index = list(map(change_index,test_df2.index))
test_df2.head(2)

In [ ]:
# map 함수

x = [1,2,3,4]

def change(x):
    return x**2

_x = list(map(change,x))
__x = list(map(lambda x:x**2,x))

print(_x)
print(__x)